In [2]:
import pandas as pd #dataframes

In [3]:
#create a dataframe for the months needed after importing the files
df = pd.DataFrame()
for month in ['03','04','05','06']:
    df = df.append(pd.read_csv("original_data/fhv_tripdata_2019-"+month+".csv" , low_memory = False), ignore_index=True)
    
        
#rename the location id to match the taxi zone location id          
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [4]:
#create a new df from selected columns
df2 = df[['LocationID', 'pickup_datetime']]

#change the datetime to date
df2['pickup_datetime'] = df2['pickup_datetime'].dt.date

#create a dataframe by merging with taxi zone
zone_lookup = pd.read_csv("original_data/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough
fhv_2019_boroughs = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0)
fhv_2019_boroughs = fhv_2019_boroughs.drop(['LocationID'], axis=1)
#fhv_2019_boroughs

<ipython-input-4-75c2df99a56c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pickup_datetime'] = df2['pickup_datetime'].dt.date


In [5]:
#create a dataframe with only usage
fhv_2019= pd.DataFrame(df2.pickup_datetime.value_counts().reset_index().values, columns=["Date", "Fhv_2019"])
fhv_2019 = fhv_2019.sort_values(by="Date")
fhv_2019

,Date,Fhv_2019
98,2019-03-01,45851
118,2019-03-02,35839
120,2019-03-03,34178
121,2019-03-04,25005
110,2019-03-05,40853
...,...,...
20,2019-06-26,71188
31,2019-06-27,69925
48,2019-06-28,67683
75,2019-06-29,56556


In [6]:
#save the dataframe as csv file
fhv_2019.to_csv("fhv_2019.csv")

In [7]:
#create a dataframe to get the usage by borough
fhv_2019_borough = fhv_2019_boroughs.groupby("Borough").count().reset_index()
fhv_2019_borough .rename(columns={'pickup_datetime':'Fhv_2019'}, inplace=True)
fhv_2019_borough

,Borough,Fhv_2019
0,Bronx,154003
1,Brooklyn,130509
2,EWR,4782
3,Manhattan,169131
4,Queens,398380
5,Staten Island,43209
6,Unknown,6345928


In [8]:
#save the dataframe as csv file
fhv_2019_borough.to_csv("fhv_2019_borough.csv")